# Importacion y generacion de matriz para algoritmos

In [1]:
import os
import numpy as np
import json
import time
from itertools import combinations, permutations
import heapq

In [2]:
"""
Se genera de manera random una nueva matriz de trabajo y se guarda en un archivo
tipo .npy
Nota: Si se requiere volver a generar una nueva matriz random, por favor eliminar el archivo
para que lo genere de nuevo, de lo contrario trabajara con el que ya existe.

"""
ruta_npy = os.path.join("data" ,"matriz_trabajo.npy")
matriz_actores = []

try:
    matriz_actores = np.load(ruta_npy)
except FileNotFoundError:
    filas = np.random.randint(10,11)
    columnas = np.random.randint(6,9)
    matriz_actores = np.random.randint(2, size=(filas, columnas))
    np.save(ruta_npy, matriz_actores)

print(matriz_actores)

[[0 0 0 1 1 0 0 0]
 [0 1 0 1 1 1 1 1]
 [1 0 0 0 0 1 1 1]
 [1 1 1 1 1 1 0 1]
 [1 0 0 0 0 0 1 1]
 [0 1 1 0 0 1 0 0]
 [0 0 0 0 1 1 1 1]
 [0 0 0 1 1 1 0 1]
 [1 0 1 0 1 1 0 0]
 [0 1 0 1 1 1 0 0]]


# Algoritmo por fuerza bruta

In [12]:
MAX_TOMAS_POR_DIA = 5
CHECKPOINT_FILE = "data/punto_guardado.json"
PROGRESO_FILE = "data/progreso.json"
TIEMPO_DESEADO_SEGUNDOS = 3600  # Tiempo límite real

"""
La matriz del ejercicio es demasiado extensa y no finaliza
por ende se recorta de acuerdo al codigo anterior y se da un maximo de tomas por dia de 4,
si se desea modificar es la variable MAX_TOMAS_POR_DIA
"""
# matriz_actores = np.array([
#     [1,1,1,1,1,0,0,0,0,0],
#     [0,0,1,1,1,0,0,0,0,0],
#     [0,1,0,0,1,0,1,0,0,0],
#     [1,1,0,0,0,0,1,1,0,0],
#     [0,1,0,1,0,0,0,1,0,0],
#     [1,1,0,1,1,0,0,0,0,0],
#     [1,1,0,1,1,0,0,0,0,0],
#     [1,1,0,0,0,1,0,0,0,0],
#     [1,1,0,1,0,0,0,0,0,0],
#     [1,1,0,0,0,1,0,0,1,0],
#     [1,1,1,0,1,0,0,1,0,0],
#     [1,1,1,1,0,1,0,0,0,0],
#     [1,0,0,1,1,0,0,0,0,0],
#     [1,0,1,0,0,1,0,0,0,0],
#     [1,1,0,0,0,0,1,0,0,0],
#     [0,0,0,1,0,0,0,0,0,1],
#     [1,0,1,0,0,0,0,0,0,0],
#     [0,0,1,0,0,1,0,0,0,0],
#     [1,0,1,0,0,0,0,0,0,0],
#     [1,0,1,1,1,0,0,0,0,0],
#     [0,0,0,0,0,1,0,1,0,0],
#     [1,1,1,1,0,0,0,0,0,0],
#     [1,0,1,0,0,0,0,0,0,0],
#     [0,0,1,0,0,1,0,0,0,0],
#     [1,1,0,1,0,0,0,0,0,1],
#     [1,0,1,0,1,0,0,0,1,0],
#     [0,0,0,1,1,0,0,0,0,0],
#     [1,0,0,1,0,0,0,0,0,0],
#     [1,0,0,0,1,1,0,0,0,0],
#     [1,0,0,1,0,0,0,0,0,0],
# ])

TOTAL_TOMAS = matriz_actores.shape[0]
tomas = list(range(TOTAL_TOMAS))

def calcular_costo(particion, matriz):
    total = 0
    for dia in particion:
        actores_dia = np.any(matriz[dia], axis=0)
        total += np.sum(actores_dia)
    return total

def generar_particiones(tomas, max_dia):
    if not tomas:
        yield []
        return
    for size in range(1, max_dia + 1):
        for grupo in combinations(tomas, size):
            restantes = [t for t in tomas if t not in grupo]
            for sub in generar_particiones(restantes, max_dia):
                yield [list(grupo)] + sub

if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, "r") as f:
        estado = json.load(f)
    mejor_costo = estado["mejor_costo"]
    mejor_particion = estado["mejor_particion"]
    checkpoint_i = estado["ultimo_index"]
else:
    mejor_costo = float('inf')
    mejor_particion = None
    checkpoint_i = 0

if os.path.exists(PROGRESO_FILE):
    with open(PROGRESO_FILE, "r") as f:
        progreso = json.load(f)
    tiempo_total = progreso["tiempo_total"]
    combinaciones_totales = progreso["combinaciones_revisadas"]
else:
    tiempo_total = 0.0
    combinaciones_totales = 0

start_time = time.time()
i = 0

for i, particion in enumerate(generar_particiones(tomas, MAX_TOMAS_POR_DIA)):
    if i < checkpoint_i:
        continue
    if time.time() - start_time >= TIEMPO_DESEADO_SEGUNDOS:
        break
    costo = calcular_costo(particion, matriz_actores)
    if costo < mejor_costo:
        mejor_costo = costo
        mejor_particion = particion

end_time = time.time()
tiempo_bloque = end_time - start_time
tiempo_total += tiempo_bloque
combinaciones_totales += (i - checkpoint_i + 1)

with open(CHECKPOINT_FILE, "w") as f:
    json.dump({
        "ultimo_index": int(i + 1),
        "mejor_costo": int(mejor_costo),
        "mejor_particion": [[int(t) for t in dia] for dia in mejor_particion]
    }, f)

with open(PROGRESO_FILE, "w") as f:
    json.dump({
        "tiempo_total": float(tiempo_total),
        "combinaciones_revisadas": int(combinaciones_totales)
    }, f)

print(f"Mejor costo hasta ahora: {mejor_costo}")
print(f"Partición óptima parcial: {mejor_particion}")
print(f"Tiempo en este flujo: {tiempo_bloque:.2f} s")
print(f"Combinaciones revisadas en este flujo: {i - checkpoint_i + 1}")
print(f"Tiempo total acumulado: {tiempo_total:.2f} s")
print(f"Combinaciones totales revisadas: {combinaciones_totales}")

Mejor costo hasta ahora: 14
Partición óptima parcial: [[0, 1, 2, 4, 6], [3, 5, 7, 8, 9]]
Tiempo en este flujo: 468.69 s
Combinaciones revisadas en este flujo: 0
Tiempo total acumulado: 25152.50 s
Combinaciones totales revisadas: 102177222


# Algoritmo Ramificacion y poda (Branch and Bound)

In [5]:
MAX_TOMAS_DIA_BB = 5
CHECKPOINT_FILE_BB = "data/punto_guardado_bb.json"
PROGRESS_FILE_BB = "data/progreso_bb.json"
TIEMPO_DESEADO_SEGUNDOS_BB = 3600

TOTAL_TOMAS_BB = matriz_actores.shape[0]
tomas_bb = list(range(TOTAL_TOMAS_BB))

class Nodo:
    def __init__(self, particion, restantes, costo):
        self.particion = particion
        self.restantes = restantes
        self.costo = costo

    def __lt__(self, other):
        return self.costo < other.costo

def calcular_costo_bb(particion, matriz):
    total = 0
    for dia in particion:
        actores_dia = np.any(matriz[dia], axis=0)
        total += np.sum(actores_dia)
    return total

def ramificacion_y_poda_bb(matriz, max_dia, tiempo_limite, start_index=0):
    mejor_costo = float('inf')
    mejor_particion = None
    heap = []
    heapq.heappush(heap, Nodo([], tomas_bb, 0))
    start_time = time.time()
    combinaciones_revisadas = 0
    index_actual = 0

    while heap:
        nodo = heapq.heappop(heap)

        if index_actual < start_index:
            index_actual += 1
            continue

        if time.time() - start_time >= tiempo_limite:
            return mejor_costo, mejor_particion, index_actual, False, combinaciones_revisadas, time.time() - start_time

        if not nodo.restantes:
            combinaciones_revisadas += 1
            costo_actual = calcular_costo_bb(nodo.particion, matriz)
            if costo_actual < mejor_costo:
                mejor_costo = costo_actual
                mejor_particion = nodo.particion
            index_actual += 1
            continue

        for size in range(1, min(max_dia, len(nodo.restantes)) + 1):
            for grupo in combinations(nodo.restantes, size):
                nuevas_restantes = [s for s in nodo.restantes if s not in grupo]
                nueva_particion = nodo.particion + [list(grupo)]
                estimacion = calcular_costo_bb(nueva_particion, matriz)
                if estimacion < mejor_costo:
                    heapq.heappush(heap, Nodo(nueva_particion, nuevas_restantes, estimacion))

        index_actual += 1

    return mejor_costo, mejor_particion, index_actual, True, combinaciones_revisadas, time.time() - start_time

if os.path.exists(CHECKPOINT_FILE_BB):
    with open(CHECKPOINT_FILE_BB, "r") as f:
        estado_bb = json.load(f)
    mejor_costo_bb = estado_bb["mejor_costo"]
    mejor_particion_bb = estado_bb["mejor_particion"]
    ultimo_index_bb = estado_bb["ultimo_index"]
    finalizado_bb = estado_bb.get("finalizado", False)

    if finalizado_bb:
        print("[Branch and Bound] Ya se han evaluado todas las combinaciones posibles.")
        exit()
else:
    mejor_costo_bb = float('inf')
    mejor_particion_bb = None
    ultimo_index_bb = 0

if os.path.exists(PROGRESS_FILE_BB):
    with open(PROGRESS_FILE_BB, "r") as f:
        progreso_bb = json.load(f)
    tiempo_total_bb = progreso_bb["tiempo_total"]
    total_combinaciones_bb = progreso_bb["combinaciones_revisadas"]
else:
    tiempo_total_bb = 0.0
    total_combinaciones_bb = 0

mejor_costo_bb_temp, mejor_particion_bb_temp, nuevo_index_bb, finalizado_bb, combinaciones_bloque_bb, tiempo_bloque_bb = ramificacion_y_poda_bb(
    matriz_actores,
    MAX_TOMAS_DIA_BB,
    TIEMPO_DESEADO_SEGUNDOS_BB,
    start_index=ultimo_index_bb
)

tiempo_total_bb += tiempo_bloque_bb
if mejor_costo_bb_temp < mejor_costo_bb:
    mejor_costo_bb = mejor_costo_bb_temp
    mejor_particion_bb = mejor_particion_bb_temp
total_combinaciones_bb += combinaciones_bloque_bb

with open(CHECKPOINT_FILE_BB, "w") as f:
    json.dump({
        "ultimo_index": int(nuevo_index_bb),
        "mejor_costo": int(mejor_costo_bb),
        "mejor_particion": [[int(t) for t in dia] for dia in mejor_particion_bb],
        "finalizado": finalizado_bb
    }, f)

with open(PROGRESS_FILE_BB, "w") as f:
    json.dump({
        "tiempo_total": float(tiempo_total_bb),
        "combinaciones_revisadas": int(total_combinaciones_bb)
    }, f)

print("[Branch and Bound]")
print(f"Mejor costo hasta ahora: {mejor_costo_bb}")
print(f"Partición óptima parcial: {mejor_particion_bb}")
print(f"Tiempo en este flujo: {tiempo_bloque_bb:.2f} s")
print(f"Combinaciones revisadas en este flujo: {combinaciones_bloque_bb}")
print(f"Tiempo total acumulado: {tiempo_total_bb:.2f} s")
print(f"Combinaciones totales revisadas: {total_combinaciones_bb}")

[Branch and Bound]
Mejor costo hasta ahora: 14
Partición óptima parcial: [[0, 3, 5, 8, 9], [1, 2, 4, 6, 7]]
Tiempo en este flujo: 465.34 s
Combinaciones revisadas en este flujo: 23495
Tiempo total acumulado: 465.34 s
Combinaciones totales revisadas: 23495
